In [ ]:
%pip install -q litellm
%pip install -q matplotlib numpy pandas httpx tqdm jupyter ipywidgets aiofiles
%pip install -q python-dotenv

In [ ]:
import os
from litellm import completion, batch_completion

from httpx import HTTPStatusError
from litellm.exceptions import NotFoundError
import pandas as pd
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from openai import OpenAI
from dotenv import load_dotenv
from typing import List

In [ ]:
load_dotenv()

In [ ]:
temp = 0.01

def comparer_prompt(responses: List[str]) -> (str, str):
    system_output = """
    You are an assistant that takes in the multiple responses from the user,
    and then writes unique pros and cons for each of the models' responses relative to other responses.

    ---
    
    Here's a basic example below. Please use this output structure:

    User input:
    [response 1]
    'hi'

    [response 2]
    'hello there'

    Your output:
    [response 1]
    Pros:
    1. shorter
    Cons:
    2. cold

    [response 2]
    Pros:
    1. more formal
    Cons:
    2. longer

    ---
    """
    user_output = ""
    for i in range(len(responses)):
        user_output += f"[response {i + 1}]\n\'" + responses[i] + "\'\n\n"

    return system_output, user_output

def judger_prompt(responses: List[str], pros_and_cons: str) -> (str, str):
    system_output = """
    You are an assistant that takes in the multiple responses and the differences (pros and cons) from the user.
    Using this, come up with the best response possible combining these responses. 

    ---
    
    Here's a basic example below. Please use this output structure:

    User input:
    [response 1]
    'hi'

    [response 2]
    'hello there'

    [pros and cons]
    [response 1]
    Pros:
    1. shorter
    Cons:
    2. cold

    [response 2]
    Pros:
    1. more formal
    Cons:
    2. longer

    Your output:
    Hi there

    ---
    """
    user_output = ""
    for i in range(len(responses)):
        user_output += f"[response {i + 1}]\n\'" + responses[i] + "\'\n\n"
    user_output += f"[pros and cons]\n" + pros_and_cons
    
    return system_output, user_output

In [ ]:
list_of_responses = []

comparer_system, comparer_user = comparer_prompt(list_of_responses)
response = completion(
          model="gpt-4.1-2025-04-14",
          messages=[
            {
                "role": "system",
                "content": comparer_system,
            },
            {
                "role": "user",
                "content": comparer_user,
            },
          ],
          temperature=temp,
    )
comparer_response = str(response.choices[0].message.content)
judger_system, judger_user = judger_prompt(list_of_responses, comparer_response)
response = completion(
          model="gpt-4.1-2025-04-14",
          messages=[
            {
                "role": "system",
                "content": judger_system,
            },
            {
                "role": "user",
                "content": judger_user,
            },
          ],
          temperature=temp,
    )
print(str(response.choices[0].message.content))